In [ ]:
from google.colab import drive #toimport google drive data

In [ ]:
drive.mount('/content/drive') #to mount the drive data into colab cloud and enterr the activation code

Mounted at /content/drive


In [ ]:
cd /content/drive/"My Drive/Colab Notebooks/TITLE ABSTRACT/"

In [ ]:
reviewer = [line.strip() for line in open("title+abstract_reviewer.txt", encoding="utf-8").readlines()]

In [ ]:
len(reviewer)

2061

In [ ]:
!pip install contextualized-topic-models

In [ ]:
from contextualized_topic_models.models.ctm import CTM
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_list, TextHandler
from contextualized_topic_models.datasets.dataset import CTMDataset
from contextualized_topic_models.utils.data_preparation import QuickText
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import common_texts
from gensim.models import ldamodel 
from contextualized_topic_models.utils.preprocessing import SimplePreprocessing
from contextualized_topic_models.utils.data_preparation import QuickText
import nltk
import os
import pickle

In [ ]:
nltk.download('stopwords')

sp = SimplePreprocessing(reviewer)
preprocessed_documents, unpreprocessed_corpus, vocab = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import spacy
import os
import re
import nltk
import string
import en_core_web_sm
import itertools, string
import numpy as np
import json
import csv
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import json
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')

#nlp = en_core_web_sm.load()
nlp=spacy.load("en_core_web_sm")

POS_KEPT = ["ADJ", "NOUN", "PROPN", "VERB"]
research_stopwords=[]
#good_tags = set(['JJ', 'JJR', 'JJS', 'NN', 'NNP', 'NNS', 'NNPS','VB','VBN','VBD','VBG','VBZ'])  #Allowed POS tags
english_stopwords = set(nltk.corpus.stopwords.words('english'))
research_stopwords=set(research_stopwords)
roman=["i","ii","iii","iv","v","vi","vii","viii","ix","x"]
roman=set(roman)
stop_words=english_stopwords.union(research_stopwords)
stop_words=stop_words.union(roman)
punct = list(string.punctuation)
punct.remove('.')
punct.remove(',')
punct.remove(':')
punct.remove(';')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import unicodedata
import operator
def remove_accents(input_str):              #function to remove greek mathematical symbols
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return (only_ascii.decode())


In [ ]:
def is_subseq(x, y):
    #if(len(x)>len(y)):
        #x,y=y,x
    it = iter(y)
    return (all(any(c == ch for c in it) for ch in x))

In [ ]:
handler = TextHandler(sentences=preprocessed_documents)
handler.prepare() # create vocabulary and training data

/usr/local/lib/python3.6/dist-packages/contextualized_topic_models/utils/data_preparation.py:161: Warning: The vocab you are using has more than 2000 words, reconstructing high-dimensional vectors requiressignificantly more training epochs and training samples. Consider reducing the number of vocabulary items. See https://github.com/MilaNLProc/contextualized-topic-models#preprocessing and https://github.com/MilaNLProc/contextualized-topic-models#tldr
  "and https://github.com/MilaNLProc/contextualized-topic-models#tldr", Warning)


In [ ]:
if os.path.exists("saved_embeddings_reviewer only new"):
  with open("saved_embeddings_reviewer only new", "rb") as filino:
    training_bert = pickle.load(filino)
else:
  training_bert = bert_embeddings_from_list(unpreprocessed_corpus, "bert-base-nli-mean-tokens", batch_size=20)      #https://pypi.org/project/spacy-sentence-bert/0.0.3/
  with open("saved_embeddings_reviewer only new", "wb") as filino:                                                                            #batch_size:size of batch to use for training
    pickle.dump(training_bert, filino)

100%|██████████| 405M/405M [00:18<00:00, 22.1MB/s]


In [ ]:
handler.bow #20000 documents with 2000 vocab in total

<2061x14020 sparse matrix of type '<class 'numpy.int64'>'
	with 116371 stored elements in Compressed Sparse Row format>

In [ ]:
training_bert.shape   #20000 document with 768 features bert embedding cell numbers     #it has tendancy to convert to 768 dimensions

(2061, 768)

In [ ]:
training_dataset = CTMDataset(handler.bow, training_bert, handler.idx2token)

In [ ]:
ctm = CTM(input_size=len(handler.vocab), bert_input_size=768, inference_type="combined", n_components=15, num_epochs=40)    #n_components: number of topic components

ctm.fit(training_dataset) 

Settings: 
                   N Components: 15
                   Topic Prior Mean: 0.0
                   Topic Prior Variance: 0.9333333333333333
                   Model Type: prodLDA
                   Hidden Sizes: (100, 100)
                   Activation: softplus
                   Dropout: 0.2
                   Learn Priors: True
                   Learning Rate: 0.002
                   Momentum: 0.99
                   Reduce On Plateau: False
                   Save Dir: None


/usr/local/lib/python3.6/dist-packages/contextualized_topic_models/models/ctm.py:47: DeprecationWarning: Direct call to CTM is deprecated and will be removed in version 2, use CombinedTM or ZeroShotTM
  warnings.warn("Direct call to CTM is deprecated and will be removed in version 2, use CombinedTM or ZeroShotTM", DeprecationWarning)


Epoch: [1/40]	Samples: [2061/82440]	Train Loss: 565.4507153403233	Time: 0:00:11.774847
Epoch: [2/40]	Samples: [4122/82440]	Train Loss: 555.2265091941866	Time: 0:00:11.235461
Epoch: [3/40]	Samples: [6183/82440]	Train Loss: 544.0019083481168	Time: 0:00:10.940606
Epoch: [4/40]	Samples: [8244/82440]	Train Loss: 538.5861504863385	Time: 0:00:10.940444
Epoch: [5/40]	Samples: [10305/82440]	Train Loss: 534.500305146167	Time: 0:00:10.888806
Epoch: [6/40]	Samples: [12366/82440]	Train Loss: 530.0398898062227	Time: 0:00:10.914559
Epoch: [7/40]	Samples: [14427/82440]	Train Loss: 527.824815064365	Time: 0:00:11.195840
Epoch: [8/40]	Samples: [16488/82440]	Train Loss: 524.8217681040757	Time: 0:00:10.973800
Epoch: [9/40]	Samples: [18549/82440]	Train Loss: 523.2810794213974	Time: 0:00:10.965823
Epoch: [10/40]	Samples: [20610/82440]	Train Loss: 521.4444269127547	Time: 0:00:10.899353
Epoch: [11/40]	Samples: [22671/82440]	Train Loss: 519.4404832302281	Time: 0:00:10.990043
Epoch: [12/40]	Samples: [24732/82440

In [ ]:
ctm.save("ctm")

/usr/local/lib/python3.6/dist-packages/contextualized_topic_models/models/ctm.py:382: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  Warning)


In [ ]:
ctm.get_topic_lists(15)

[['gate',
  'design',
  'element',
  'simulation',
  'numerical',
  'compact',
  'switching',
  'shell',
  'circuit',
  'transistor',
  'charge',
  'conical',
  'displacement',
  'motor',
  'vibration'],
 ['dielectric',
  'gate',
  'transistor',
  'beam',
  'polarization',
  'charge',
  'oxide',
  'band',
  'device',
  'dual',
  'thickness',
  'silicon',
  'range',
  'material',
  'design'],
 ['the',
  'image',
  'art',
  'pixel',
  'segmentation',
  'feature',
  'processing',
  'coefficient',
  'object',
  'vision',
  'step',
  'classification',
  'automatic',
  'statistical',
  'transform'],
 ['project',
  'management',
  'environment',
  'development',
  'industry',
  'significant',
  'various',
  'health',
  'economy',
  'social',
  'government',
  'natural',
  'change',
  'water',
  'global'],
 ['service',
  'mobile',
  'cloud',
  'network',
  'infrastructure',
  'security',
  'smart',
  'protocol',
  'computing',
  'attack',
  'sensor',
  'issue',
  'environment',
  'communicatio

In [ ]:
import pandas as pd
b=ctm.get_doc_topic_distribution(training_dataset,20)            #TO FIND DOC-TOPIC DISTRIBUTION
DF = pd.DataFrame(b) 
DF.to_csv("similarity_Score_reviewer.csv")

In [ ]:
pip install matplotlib==3.1.3

     |████████████████████████████████| 13.1MB 319kB/s 
ERROR: contextualized-topic-models 1.7.1 has requirement matplotlib==3.3.3, but you'll have matplotlib 3.1.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.3.3
    Uninstalling matplotlib-3.3.3:
      Successfully uninstalled matplotlib-3.3.3


/usr/local/lib/python3.6/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.6/dist-packages/matplotlib-3.1.3.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):


In [ ]:
ctm.get_wordcloud(topic_id=8, n_words=25)

In [ ]:
query = [line.strip() for line in open("title+abstract_query.txt", encoding="utf-8").readlines()]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ResourceWarning: unclosed file <_io.TextIOWrapper name='title+abstract_query.txt' mode='r' encoding='utf-8'>
  """Entry point for launching an IPython kernel.


In [ ]:
nltk.download('stopwords')

sp = SimplePreprocessing(query)
preprocessed_documents_query, unpreprocessed_corpus_query, vocab = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/corpus/reader/wordlist.py:28: ResourceWarning: unclosed file <_io.BufferedReader name='/root/nltk_data/corpora/stopwords/english'>
  return concat([self.open(f).read() for f in fileids])


In [ ]:
qt=QuickText("bert-base-nli-mean-tokens",text_for_bert=unpreprocessed_corpus_query,text_for_bow=preprocessed_documents_query)

In [ ]:
len(unpreprocessed_corpus_query)

611

In [ ]:
testing_dataset=qt.load_dataset()

In [ ]:
testing_dataset[0]['X_bert'].shape

torch.Size([768])

In [ ]:
ctm.get_thetas(testing_dataset, n_samples=20) # returns a (n_documents, n_topics) matrix with the topic distribution of each document

/usr/local/lib/python3.6/dist-packages/contextualized_topic_models/models/ctm.py:254: DeprecationWarning: Call to `get_thetas` is deprecated and will be removed in version 2, use `get_doc_topic_distribution` instead
  DeprecationWarning)


RuntimeError: ignored

In [ ]:
ctm.get_predicted_topics(testing_dataset,n_samples=20)